In [4]:
import requests
import numpy as np
import pandas as pd
import requests,re,io
from bs4 import BeautifulSoup

In [6]:
state_links = {"New York":"https://fred.stlouisfed.org/release/tables?rid=175&eid=267974"}

main_list = []

for key,value in state_links.items():
    res = requests.get(value)
    #print(key)
    html_content = res.text

    tr_tags = re.findall(r'<tr[^>]*>(.*?)</tr>', html_content, re.DOTALL)
    for tr in tr_tags:
        a_tag = re.search(r'<a[^>]*href="(.*?)"[^>]*>(.*?)</a>', tr, re.DOTALL)
        if a_tag:
            baseurl = "https://fred.stlouisfed.org"
            state = key
            county=a_tag.group(2)
            pid = re.search(r'/series/(\w+)', a_tag.group(1)).group(1)
            csv_url = "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=748&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id={0}&scale=left&cosd=1969-01-01&coed=2021-01-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Annual&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2023-02-10&revision_date=2023-02-10&nd=1969-01-01".format(pid)
            #print(baseurl+a_tag.group(1))
            #download the csv file of each county for a state
            print(state,county,pid)
            res = requests.get(csv_url).content

            df = pd.read_csv(io.StringIO(res.decode('utf-8')))
            # rename the county name 
            df = df.rename(columns={pid:"per_capita_income"})
            
            df['county'] = county
            df['state'] = state
            
            # now combine all the df into one so that for each staate and each county all th pci will be stored
            main_list.append(df)

            
df = pd.concat(main_list,axis=0, ignore_index=True)

df.to_csv("NY_per_county_capita_income.csv") 

New York Albany County PCPI36001
New York Allegany County PCPI36003
New York Bronx County PCPI36005
New York Broome County PCPI36007
New York Cattaraugus County PCPI36009
New York Cayuga County PCPI36011
New York Chautauqua County PCPI36013
New York Chemung County PCPI36015
New York Chenango County PCPI36017
New York Clinton County PCPI36019
New York Columbia County PCPI36021
New York Cortland County PCPI36023
New York Delaware County PCPI36025
New York Dutchess County PCPI36027
New York Erie County PCPI36029
New York Essex County PCPI36031
New York Franklin County PCPI36033
New York Fulton County PCPI36035
New York Genesee County PCPI36037
New York Greene County PCPI36039
New York Hamilton County PCPI36041
New York Herkimer County PCPI36043
New York Jefferson County PCPI36045
New York Kings County PCPI36047
New York Lewis County PCPI36049
New York Livingston County PCPI36051
New York Madison County PCPI36053
New York Monroe County PCPI36055
New York Montgomery County PCPI36057
New Yor